# Python 2 HSUTCC: Session 12: Modules & Packages

Modules are a fundamental concept in Python that help organize and structure code. They allow you to break down large programs into smaller, more manageable pieces.

## What is a Module?

In Python, a module is simply a file with a .py extension containing Python code. Each module creates its own namespace, which corresponds to names defined in the file.

## Creating a Module

Let's create a simple module called `useful.py`

```python
# useful.py
"""I'm a useful module."""

some_variable = "foobar"

def boo() -> None:
    return 42
```

## Importing Modules

### Using the 'import' statement

In [1]:
import useful

print(useful.some_variable)
print(useful.boo())

foobar
42


One thing to note is that module imported will be cached, so importing it again will not actually reload the whole module. Try edit the `useful.py` by adding
```python
new_variable = "I am brand new!"
```

In [1]:
import useful

print(useful.new_variable)

I am new!


### Using 'from ... import'

In [1]:
from useful import boo, some_variable

print(some_variable)
print(boo())

foobar
42


### Using 'import ... as'

In [1]:
import useful as u

print(u.some_variable)
print(u.boo())

foobar
42


### Sidenotes: Namespace, long time no see...

When we discuss about function, we did talk about it having its own pocket of namespace. In fact, we forget to talk about namespace when talking about class. When we are defining a class.

In [2]:
class HSStudent():
    def __init__(self, name) -> None:
        self.name = name

    def beg_for_less_hw(self):
        print('Less homework please~')

That attribute and method live in its own class namespace, easy way to see that is by us using dot notation.

In [3]:
student = HSStudent('Tom')

In [5]:
student.name

'Tom'

In [6]:
student.beg_for_less_hw()

Less homework please~


Anything defined in one module is also live in a seperate world from another. For examples

```python
# do_many_things_1.py
x = 20

# do_many_things_2.py
x = 35
```

In [7]:
import do_many_things_1
import do_many_things_2

print(do_many_things_1.x, do_many_things_2.x)

20 35


In [8]:
print(x)

NameError: name 'x' is not defined

Unless... you make the two world collide yourself

In [9]:
from do_many_things_1 import x
from do_many_things_2 import x

print(x)

35


## Module Attributes

When you import a module, it comes with some built-in attributes:

In [1]:
import useful

print(useful.__name__)
print(useful.__doc__)
print(useful.__file__)

useful
I'm a useful module.
/Users/ruji_mac/Library/Mobile Documents/com~apple~CloudDocs/Work/HSUTCC/Teaching/python1-2025/Python2/module_session/useful.py


In [3]:
import math

print(math.__file__)

/opt/anaconda3/envs/locth-lesson/lib/python3.13/lib-dynload/math.cpython-313-darwin.so


## Executing Modules as Scripts

You can also run a module as a standalone script. When a module is run directly, its **name** variable is set to "**main**". You can use this to include code that should only run when the module is executed directly:

```python
# useful.py

...

def test():
    assert boo() == 42

if __name__ == "__main__":
    print("Running tests...")
    test()
    print("OK")
```

You will be able to try this in your terminal.

In [1]:
import useful

I am outside.


## Many modules is a package

To organize your code more neatly (than you room, I hope), we create a package which is a folder of multiple modules.

```
package_a
├── __init__.py
├── boo.py
└── foo.py
```

Note that there is this `__init__.py` file popping up from somewhere. This is not a must but a very recommended way to create a **regular package** (we will not discuss the **namespace package** in this class). Now, we can import our module like so.

In [2]:
import package_a.foo
package_a.foo.foo_greet()

foo imported
bar imported
boo imported
Hello


In [1]:
from package_a import foo

foo.foo_greet()

foo imported
bar imported
boo imported
Hello


In [1]:
from package_a.foo import foo_greet

foo_greet()

foo imported
bar imported
boo imported
Hello


### Relative Import

Now, when you are a little file inside a package and want to use another file, it is recommended to use relative import (though PEP8 disagree on that note).

```python
# foo.py
def foo_greet():
    print('Hello')

# bar.py
from . import foo
```

In [2]:
import package_a.bar as bar

bar.bar_greet()

Hello


If you have subpackages, you can traverse through the directory to find the targeted one.

```
package_b
├── __init__.py
├── calculator.py
└── package_c
    ├── __init__.py
    └── foo.py
```

In [3]:
from package_b.package_c import foo

foo.foo_subtract()

foo imported


2

In [ ]:
import package_b.package_c.foo

package_b.package_c.foo.foo_subtract()

So far so good, but remembering the package structure in order to work with a module can become troublesome for the user. How can we make it user-freindly? Answer: **encapsulation**.

### `__init__.py`

To hide those structural detail, we can use `__init__` file to aggregate everything together.

```
package_a
├── __init__.py
├── bar.py
├── boo.py
└── foo.py
```

```python
# __init__.py
from .foo import *
from .bar import *
from .boo import *
```

In [4]:
from package_a.foo import foo_greet
from package_a.bar import bar_greet

foo_greet()

Hello


In [1]:
from package_a import foo_greet, bar_greet

foo_greet()
bar_greet()

foo imported
bar imported
boo imported
Hello
Hello


We have discussed that `import *` is to be used with caution because it will pollute your namepace. Is there anyway that as a library creator, we make it less problematic? Turn out you can control what can be imported from a module with the help of the `__all__` variable.

```python
# foo.py
def foo_greet():
    print('Hello')

__all__ = []

# boo.py
def boo_greet():
    print('Hello')

__all__ = ['boo_greet']

# __init__.py
from .foo import *
from .bar import *
from .boo import *

__all__ = foo.__all__ + boo.__all__
```

In [1]:
from package_a import *

foo_greet()

foo imported
bar imported
boo imported


NameError: name 'foo_greet' is not defined

In [2]:
boo_greet()

Hello


This also makes sure that the user won't import some internal artifact in from your library.

## I am sure that I have installed the module, why can't I import!

The rule is, if it's not in the path, it won't import.

In [ ]:
import sys
sys.path

['/opt/anaconda3/envs/locth-lesson/lib/python313.zip',
 '/opt/anaconda3/envs/locth-lesson/lib/python3.13',
 '/opt/anaconda3/envs/locth-lesson/lib/python3.13/lib-dynload',
 '',
 '/opt/anaconda3/envs/locth-lesson/lib/python3.13/site-packages']

In [4]:
import mytest

ModuleNotFoundError: No module named 'mytest'

In [5]:
sys.path.append('/Users/ruji_mac/Library/Mobile Documents/com~apple~CloudDocs/Work/HSUTCC/Teaching/python1-2025')

In [6]:
sys.path

['/opt/anaconda3/envs/locth-lesson/lib/python313.zip',
 '/opt/anaconda3/envs/locth-lesson/lib/python3.13',
 '/opt/anaconda3/envs/locth-lesson/lib/python3.13/lib-dynload',
 '',
 '/opt/anaconda3/envs/locth-lesson/lib/python3.13/site-packages',
 '/Users/ruji_mac/Library/Mobile Documents/com~apple~CloudDocs/Work/HSUTCC/Teaching/python1-2025']

In [7]:
import mytest

test imported


So if you want to troubleshoot your module not importing, here is one place to look.

### Namespace Package

There is one more namespace that we haven't covered and actually don't want to cover in such detail, but to give you a glimse, here we go.

Say now we have the following packages
```
package_d
└── spam
    └── foo.py
package_e
└── spam
    └── bar.py
```
Note that these packages have no `__init__` files.

In [8]:
import package_d.spam.foo
import package_e.spam.bar

foo imported
bar imported


However, if you are to appended these directories to the `path`.

In [1]:
import sys
sys.path.extend(['package_d', 'package_e'])
sys.path

['/opt/anaconda3/envs/locth-lesson/lib/python313.zip',
 '/opt/anaconda3/envs/locth-lesson/lib/python3.13',
 '/opt/anaconda3/envs/locth-lesson/lib/python3.13/lib-dynload',
 '',
 '/opt/anaconda3/envs/locth-lesson/lib/python3.13/site-packages',
 'package_d',
 'package_e']

In [3]:
import spam.foo
import spam.bar

foo imported
bar imported


Looking at the path of the package, you can see that the namespace consist of two differences path.

In [4]:
spam.__path__

_NamespacePath(['/Users/ruji_mac/Library/Mobile Documents/com~apple~CloudDocs/Work/HSUTCC/Teaching/python1-2025/Python2/module_session/package_d/spam', '/Users/ruji_mac/Library/Mobile Documents/com~apple~CloudDocs/Work/HSUTCC/Teaching/python1-2025/Python2/module_session/package_e/spam'])

In [5]:
for path in spam.__path__:
    print(path)

/Users/ruji_mac/Library/Mobile Documents/com~apple~CloudDocs/Work/HSUTCC/Teaching/python1-2025/Python2/module_session/package_d/spam
/Users/ruji_mac/Library/Mobile Documents/com~apple~CloudDocs/Work/HSUTCC/Teaching/python1-2025/Python2/module_session/package_e/spam


### Sidenote: what is really a module?

Since everything in Python is an object, hence module is also an object as well.

In [6]:
from package_a import foo

print(type(foo))

foo imported
bar imported
boo imported
<class 'module'>


More importantly, module is a dictionary. And you can see this if you implement it from scratch.

In [7]:
import types

my_module = types.ModuleType('harbourspace')
my_module.__dict__

{'__name__': 'harbourspace',
 '__doc__': None,
 '__package__': None,
 '__loader__': None,
 '__spec__': None}

In [8]:
foo.__dict__

{'__name__': 'package_a.foo',
 '__doc__': None,
 '__package__': 'package_a',
 '__loader__': <_frozen_importlib_external.SourceFileLoader at 0x105f45a90>,
 '__spec__': ModuleSpec(name='package_a.foo', loader=<_frozen_importlib_external.SourceFileLoader object at 0x105f45a90>, origin='/Users/ruji_mac/Library/Mobile Documents/com~apple~CloudDocs/Work/HSUTCC/Teaching/python1-2025/Python2/module_session/package_a/foo.py'),
 '__file__': '/Users/ruji_mac/Library/Mobile Documents/com~apple~CloudDocs/Work/HSUTCC/Teaching/python1-2025/Python2/module_session/package_a/foo.py',
 '__cached__': '/Users/ruji_mac/Library/Mobile Documents/com~apple~CloudDocs/Work/HSUTCC/Teaching/python1-2025/Python2/module_session/package_a/__pycache__/foo.cpython-313.pyc',
 '__builtins__': {'__name__': 'builtins',
  '__doc__': "Built-in functions, types, exceptions, and other objects.\n\nThis module provides direct access to all 'built-in'\nidentifiers of Python; for example, builtins.len is\nthe full name for the bui

When you import a module, 3 steps are happening behind the scene.

1. Python check wheter there is a module in the cache. If it's there, it will use that cache without reloading the module.

In [1]:
import sys
sys.modules

{'sys': <module 'sys' (built-in)>,
 'builtins': <module 'builtins' (built-in)>,
 '_frozen_importlib': <module '_frozen_importlib' (frozen)>,
 '_imp': <module '_imp' (built-in)>,
 '_thread': <module '_thread' (built-in)>,
 '_warnings': <module '_warnings' (built-in)>,
 '_weakref': <module '_weakref' (built-in)>,
 '_io': <module '_io' (built-in)>,
 'marshal': <module 'marshal' (built-in)>,
 'posix': <module 'posix' (built-in)>,
 '_frozen_importlib_external': <module '_frozen_importlib_external' (frozen)>,
 'time': <module 'time' (built-in)>,
 'zipimport': <module 'zipimport' (frozen)>,
 '_codecs': <module '_codecs' (built-in)>,
 'codecs': <module 'codecs' (frozen)>,
 'encodings.aliases': <module 'encodings.aliases' from '/opt/anaconda3/envs/locth-lesson/lib/python3.13/encodings/aliases.py'>,
 'encodings': <module 'encodings' from '/opt/anaconda3/envs/locth-lesson/lib/python3.13/encodings/__init__.py'>,
 'encodings.utf_8': <module 'encodings.utf_8' from '/opt/anaconda3/envs/locth-lesson/l

In [2]:
import package_a
sys.modules

foo imported
bar imported
boo imported


{'sys': <module 'sys' (built-in)>,
 'builtins': <module 'builtins' (built-in)>,
 '_frozen_importlib': <module '_frozen_importlib' (frozen)>,
 '_imp': <module '_imp' (built-in)>,
 '_thread': <module '_thread' (built-in)>,
 '_warnings': <module '_warnings' (built-in)>,
 '_weakref': <module '_weakref' (built-in)>,
 '_io': <module '_io' (built-in)>,
 'marshal': <module 'marshal' (built-in)>,
 'posix': <module 'posix' (built-in)>,
 '_frozen_importlib_external': <module '_frozen_importlib_external' (frozen)>,
 'time': <module 'time' (built-in)>,
 'zipimport': <module 'zipimport' (frozen)>,
 '_codecs': <module '_codecs' (built-in)>,
 'codecs': <module 'codecs' (frozen)>,
 'encodings.aliases': <module 'encodings.aliases' from '/opt/anaconda3/envs/locth-lesson/lib/python3.13/encodings/aliases.py'>,
 'encodings': <module 'encodings' from '/opt/anaconda3/envs/locth-lesson/lib/python3.13/encodings/__init__.py'>,
 'encodings.utf_8': <module 'encodings.utf_8' from '/opt/anaconda3/envs/locth-lesson/l

In [ ]:
import sys
import package_a
from package_a import foo
sys.modules

{'sys': <module 'sys' (built-in)>,
 'builtins': <module 'builtins' (built-in)>,
 '_frozen_importlib': <module '_frozen_importlib' (frozen)>,
 '_imp': <module '_imp' (built-in)>,
 '_thread': <module '_thread' (built-in)>,
 '_warnings': <module '_warnings' (built-in)>,
 '_weakref': <module '_weakref' (built-in)>,
 '_io': <module '_io' (built-in)>,
 'marshal': <module 'marshal' (built-in)>,
 'posix': <module 'posix' (built-in)>,
 '_frozen_importlib_external': <module '_frozen_importlib_external' (frozen)>,
 'time': <module 'time' (built-in)>,
 'zipimport': <module 'zipimport' (frozen)>,
 '_codecs': <module '_codecs' (built-in)>,
 'codecs': <module 'codecs' (frozen)>,
 'encodings.aliases': <module 'encodings.aliases' from '/opt/anaconda3/envs/locth-lesson/lib/python3.13/encodings/aliases.py'>,
 'encodings': <module 'encodings' from '/opt/anaconda3/envs/locth-lesson/lib/python3.13/encodings/__init__.py'>,
 'encodings.utf_8': <module 'encodings.utf_8' from '/opt/anaconda3/envs/locth-lesson/l

2. When importing a new module, python compile the soucecode into a bytecode.

In [3]:
code = 'for index in range(10): print(index)'
result = compile(code, 'result.py', mode='exec')

In [4]:
result

<code object <module> at 0x107ccdc30, file "result.py", line 1>

In [5]:
import dis
dis.dis(result)

  0           RESUME                   0

  1           LOAD_NAME                0 (range)
              PUSH_NULL
              LOAD_CONST               0 (10)
              CALL                     1
              GET_ITER
      L1:     FOR_ITER                11 (to L2)
              STORE_NAME               1 (index)
              LOAD_NAME                2 (print)
              PUSH_NULL
              LOAD_NAME                1 (index)
              CALL                     1
              POP_TOP
              JUMP_BACKWARD           13 (to L1)
      L2:     END_FOR
              POP_TOP
              RETURN_CONST             1 (None)


3. Python then executes the code, and cache the result.

In [6]:
exec(result)

0
1
2
3
4
5
6
7
8
9


Now, let's wrap everything up with the story of the cyclic import.

```python
# package_f/foo.py
print('foo imported')

from . import bar

def foo_greet():
    print('Hello from foo')

# package_f/bar.py
print('bar imported')

from . import foo
```

In [7]:
import package_f.foo

foo imported
bar imported


Let's try adding this code to `bar.py`

```python
foo_greet()
```

In [1]:
import package_f.foo

foo imported
bar imported
Hello from foo


Anyone want to learn more? Check a tutorial from David Beazley from Pycon 2015 at https://youtu.be/0oTh1CXRaQ0?si=W3CDyJnxCEs7U35G.